# 微信聊天记录的数据分析

* Slug: wechat_analyze

* Date: 2018-05-27 16:00

* Category: 拾遗

* Tags:

* Author: timking

* Summary: 简单的展示了一个分析微信聊天记录的流程


某一天突发奇想，想要把聊天记录中和某人的聊天导出来做一轮数据分析。所以就开始了这么一个邪恶的计划 :)

由于前人已经破解了微信的加密规则，在目前加密模式还没变的情况下，只需要参照前人的经验就好了。但是在目前微信数亿用户的情况下，而本地数据库又是存在用户的手机上，微信不太可能对此进行升级，所以密码的加密规则也不太可能改变。如果对反编译的过程有兴趣，可以参照以下文章。

> [反编译微信参考](http://blog.csdn.net/jiangwei0910410003/article/details/52238891)

## 导出微信的数据库文件

已知微信消息的存放地址在 `/data/data/com.tencent.mm/MicroMsg/<md5hash>/EnMicroMsg.db` 中。  

找到之后，可以通过 `adb pull /data/data/com.tencent.mm/MicroMsg/<md5hash>/EnMicroMsg.db d:/` 命令将数据库文件导出到本地。  

这里需要注意的有两点。

* md5hash的值：md5(mm+uin)，值得注意的是，在第二步没有执行之前是并没有办法知道UIN的值的，这里最简单的就是直接看像md5串的目录名。再通过第二步就可以知道具体的UIN值了。
* 如果发现目录打不开或者data目录下没有内容。可以通过adb工具添加权限。具体执行如下：  

```shell
adb shell
su
chmod 777 /data
chmod 777 /data/data
chmod -R 777 /data/data/com.tencent.mm
```

## 获取解密用的key

已知微信本地数据库的加密规则为 key=md5(IMEI+uin) | cut -c -7，这里详细介绍一下如何获取IMEI和UIN。 

获取IMEI:   
有些手机可以直接从系统信息上看到IMEI（比如说MEIZU），如果看不到的，可以通过在拨号盘输入 `*#06#` 查看获取。

获取UIN:   
UIN在 com.tencent.mm/shared_prefs/system_config_prefs.xml 中，找到 default_uin 条目，该值由于int32的溢出，可能为负数。

## 解密微信的数据库文件

对数据库进行解密。下载并安装sqlcipher。  

Mac下可以通过 `brew install sqlcipher` 进行安装。  
注意下载的版本为3.x，但是目前微信使用的是2.x的加密方式，而3.x默认不向下兼容，需要手动指定。  

执行以下命令进行解密数据库。然后在同级目录下会出现decrypted_database.db文件。  
至此解密数据库完成，可以通过sqlite3数据库进行读取数据。
```bash
sqlcipher EnMicroMsg.db
PRAGMA key = "<第二步拿到的key>";
PRAGMA cipher_migrate;
ATTACH DATABASE "decrypted_database.db" AS decrypted_database KEY "";
SELECT sqlcipher_export("decrypted_database");
DETACH DATABASE decrypted_database;
```

## 获取和聊天对象的聊天记录

上一步获取到解密的数据库之后，就可以使用 `sqllite` 工具读取数据库文件了。  

可以通过以下sql语句查询出与某人的聊天记录。查询出具体的数据之后导成csv。  
```sql
select datetime(subStr(cast(m.createTime as text),1,10),'unixepoch', 'localtime') as theTime,case m.isSend when 0 then r.nickname when 1 then '我'end as person,m.content from message m inner join rcontact r on m.talker = r.username where m.type=1 and r.nickname = '<微信昵称>'
```

## 对聊天记录进行处理

到了这一步总算进入正题了，通过上面拿到了我们想要的聊天记录。  
现在就可以进行我们想要做的分析啦。 :)

粗略的看了一下导出的csv文件。   
csv文件是以发言时间、发送人、发送文本的格式组成的。由`\t`分割开。  
同时在聊天对话中会存在很多换行符，所以需要对换行符进行处理。 


未完待续。。。

## 参考文章:
> https://blog.slinuxer.com/2015/10/微信聊天记录  
http://blog.csdn.net/njweiyukun/article/details/54024442